In [2]:
import numpy as np
import pprint as pp

Defining matrix A and tolerance

In [3]:
A = np.array([[-30.,10.,20.],[10.,40.,-50.],[20.,-50.,-10.]])
n = len(A)
tolerance = 1E-9

Helper function that calculates eigenvalues - (Rayleigh's quotient)

In [16]:
def eigenvalue(A,v):
    Av = A.dot(v)
    return v.dot(Av)/v.dot(v)


### Power method for finding the largest eigenvalue

In [17]:
def power_iteration(A):
    """
    Converges to eigenvalue farthest from zero and associated eigenvector.
    Procedure:
    1. Let v be an approximation to x1 (random vector of unit magnitude)
    2. Compute vector z = Av
    3. Compute |z|
    4. Let v = z/|z| and repeat 2-4 until change in v is negligible.
    """
    n,d = A.shape
    v = np.ones(d) / np.sqrt(d)
    ev = eigenvalue(A,v)
    
    iteration = 1
    while True:
        Av = A.dot(v)
        v_new = Av / np.linalg.norm(Av)
        
        ev_new = eigenvalue(A,v_new)
        if np.abs(ev - ev_new) < tolerance:
            break
        print("Iteration #:",iteration)
        print("Eigenvalue:","%.6f"%ev_new)
        print("Eigenvector:",end=' ')
        pp.pprint(v_new)
        print('\n')
        v = v_new
        ev = ev_new
        iteration += 1
        
    print("\n\nLargest eigenvalue: %.6f"%ev_new)
    print("Took %d iterations"%(iteration-1))
    return ev_new, v_new

In [15]:
_eigenvalue, _eigenvector = power_iteration(A)

Iteration #: 1
Eigenvalue: -10.000000
Eigenvector: array([ 0.,  0., -1.])


Iteration #: 2
Eigenvalue: 3.000000
Eigenvector: array([-0.36514837,  0.91287093,  0.18257419])


Iteration #: 3
Eigenvalue: 15.726979
Eigenvector: array([ 0.36947327,  0.36947327, -0.85263063])


Iteration #: 4
Eigenvalue: 27.577558
Eigenvector: array([-0.37111048,  0.9277762 , -0.03883714])


Iteration #: 5
Eigenvalue: 38.004319
Eigenvector: array([ 0.29306512,  0.52751722, -0.79739477])


Iteration #: 6
Eigenvalue: 46.632560
Eigenvector: array([-0.28638499,  0.94018052, -0.18451078])


Iteration #: 7
Eigenvalue: 53.411719
Eigenvector: array([ 0.2080184 ,  0.63946397, -0.7401447 ])


Iteration #: 8
Eigenvalue: 58.524038
Eigenvector: array([-0.21115374,  0.93212122, -0.29421783])


Iteration #: 9
Eigenvalue: 62.261356
Eigenvector: array([ 0.13992008,  0.71430432, -0.68570527])


Iteration #: 10
Eigenvalue: 64.931852
Eigenvector: array([-0.15346284,  0.915713  , -0.37137428])


Iteration #: 11
Eigenvalue: 66.80

In [18]:
print("Largest eigenvalue: %.6f"%_eigenvalue)

Largest eigenvalue: 70.943483


Thus, the **power method** took 66 iterations to converge to the largest eigenvalue, 70.943483, for given tolerance

<br><br> 
### Inverse power method for finding smallest eigenvalue

Implementing helper LU decomposition functions

In [19]:
def LUdecomp(a):
    """
    a = LUdecomp(a)
    LU decomposition: [L][U] = [a]
    The returned matrix [a] = [L-U] contains [U] in the upper triangle and 
    the nondiagonal terms of [L] in the lower triangle.
    """
    n = len(a)
    for k in range(0,n-1):
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lam = a[i,k]/a[k,k]
                a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
                a[i,k] = lam
    return a

def LUsolve(a,b):
    """x = LUsolve(a,b)
    Solves [L][U]{x} = b, 
    where [a] = [L\\U] is the matrix returnedfrom LUdecomp
    """
    n = len(a)
    for k in range(1,n):
        b[k] = b[k] - np.dot(a[k,0:k],b[0:k])
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
    return b

Inverse power method implementation

In [20]:
from math import sqrt
from random import random

def inverse_power(a,s,maxIter,tol=1E-9):
    """
    Inverse power method for solving the eigenvalue problem 
    [a]{x} = lam{x}.
    Returns 'lam' closest to 's' and the corresponding eigenvector {x}
    's' allows the method of eigenvalue shifting - corresponding convergence
    is much faster.
    """
    n = len(a)
    
    #form [a*] = [a] - s[I]
    a_star = a - np.identity(n)*s
    #decompose [a*]
    a_star = LUdecomp(a_star)
    
    x = np.zeros((n),dtype=float)
    
    #seed [x] with random numbers
    for i in range(n):
        x[i] = random()
    
    #normalize [x]
    x_mag = np.linalg.norm(x)
    x = x/x_mag
    
    for i in range(maxIter):
        #save current x
        x_old = np.copy(x)
        #solve [a*][x] = [xOld]
        x = LUsolve(a_star,x)
        
        #normalize [x]
        x_mag = np.linalg.norm(x)
        
        x = x/x_mag
        
        #detect change in sign of [x]
        if np.dot(x_old,x)<0.0:
            sign = -1.0
            x = -x
        else:
            sign = 1.0
        
        print('Iteration #',i+1)
        print('Eigenvalue: %.6f'%(s+sign/x_mag))
        print('Eigenvector:',end=' ')
        pp.pprint(x)
        print('\n')
        
        if np.linalg.norm(x_old-x) < tol:
            return s+sign/x_mag, x
    
    print('Inverse power method did not converge')
    

In [22]:
small_eval,s_evector = inverse_power(A,0,100)

Iteration # 1
Eigenvalue: -17.181693
Eigenvector: array([0.64823358, 0.37626692, 0.66197918])


Iteration # 2
Eigenvalue: -12.808236
Eigenvector: array([0.75841768, 0.36493864, 0.54002076])


Iteration # 3
Eigenvalue: -12.564789
Eigenvector: array([0.78043842, 0.3433804 , 0.52249954])


Iteration # 4
Eigenvalue: -12.553583
Eigenvector: array([0.7849763 , 0.34183937, 0.51667984])


Iteration # 5
Eigenvalue: -12.553070
Eigenvector: array([0.78595991, 0.34094776, 0.51577286])


Iteration # 6
Eigenvalue: -12.553047
Eigenvector: array([0.78616869, 0.34085467, 0.51551613])


Iteration # 7
Eigenvalue: -12.553046
Eigenvector: array([0.78621399, 0.34081718, 0.51547183])


Iteration # 8
Eigenvalue: -12.553046
Eigenvector: array([0.78622366, 0.34081221, 0.51546037])


Iteration # 9
Eigenvalue: -12.553046
Eigenvector: array([0.78622575, 0.3408106 , 0.51545825])


Iteration # 10
Eigenvalue: -12.553046
Eigenvector: array([0.78622619, 0.34081035, 0.51545774])


Iteration # 11
Eigenvalue: -12.553046
E

In [23]:
print("Smallest eigenvalue using inverse power iterations: %.6f"%small_eval)
print("Corresponding eigenvector:")
pp.pprint(s_evector)

Smallest eigenvalue using inverse power iterations: -12.553046
Corresponding eigenvector:
array([0.78622632, 0.34081026, 0.51545761])


Therefore, **inverse power method** took 15 iterations to converge to the smallest eigenvalue, -12.553046, for given tolerance

*Pentadiagonal matrix example*

Generating matrix from 3 smaller matrices a, b, c

In [24]:
def pentadiag(a,b,c):
    return np.diag(a,-2) + np.diag(b,-1) + np.diag(c,0) + np.diag(b,1) + np.diag(a,2)

a = np.ones((8),dtype=int)
b = 2*np.ones((9),dtype=int)
c = 4*np.ones((10),dtype=int)
A2 = pentadiag(a,b,c)
A2

array([[4, 2, 1, 0, 0, 0, 0, 0, 0, 0],
       [2, 4, 2, 1, 0, 0, 0, 0, 0, 0],
       [1, 2, 4, 2, 1, 0, 0, 0, 0, 0],
       [0, 1, 2, 4, 2, 1, 0, 0, 0, 0],
       [0, 0, 1, 2, 4, 2, 1, 0, 0, 0],
       [0, 0, 0, 1, 2, 4, 2, 1, 0, 0],
       [0, 0, 0, 0, 1, 2, 4, 2, 1, 0],
       [0, 0, 0, 0, 0, 1, 2, 4, 2, 1],
       [0, 0, 0, 0, 0, 0, 1, 2, 4, 2],
       [0, 0, 0, 0, 0, 0, 0, 1, 2, 4]])

**Largest eigenvalue, using power method**

In [25]:
_eigenvalue2, _eigenvector2 = power_iteration(A2)

Iteration #: 1
Eigenvalue: 9.423256
Eigenvector: array([0.23869802, 0.30689745, 0.34099717, 0.34099717, 0.34099717,
       0.34099717, 0.34099717, 0.34099717, 0.30689745, 0.23869802])


Iteration #: 2
Eigenvalue: 9.495542
Eigenvector: array([0.20217265, 0.28881807, 0.34297146, 0.35741237, 0.36102259,
       0.36102259, 0.35741237, 0.34297146, 0.28881807, 0.20217265])


Iteration #: 3
Eigenvalue: 9.526969
Eigenvector: array([0.18193768, 0.27385609, 0.33956636, 0.36691398, 0.37678952,
       0.37678952, 0.36691398, 0.33956636, 0.27385609, 0.18193768])


Iteration #: 4
Eigenvalue: 9.541698
Eigenvector: array([0.16944427, 0.26285381, 0.33558095, 0.37256221, 0.38830315,
       0.38830315, 0.37256221, 0.33558095, 0.26285381, 0.16944427])


Iteration #: 5
Eigenvalue: 9.548694
Eigenvector: array([0.16126394, 0.25503846, 0.33224856, 0.37607514, 0.39639337,
       0.39639337, 0.37607514, 0.33224856, 0.25503846, 0.16126394])


Iteration #: 6
Eigenvalue: 9.552019
Eigenvector: array([0.15575198, 0.

In [27]:
print("Largest eigenvalue: %.6f"%_eigenvalue2)

Largest eigenvalue: 9.555026


Therefore, **power method** takes 26 iterations to obtain largest eigenvalue, 9.555026, for given tolerance.

<br>**Smallest eigenvalue, using inverse power method**

Running first with 0 shift, to get an estimate of smallest value

In [28]:
inverse_power(A2,0,100,tol=1E-6)

Iteration # 1
Eigenvalue: 5.030621
Eigenvector: array([ 0.5521873 ,  0.24011326, -0.37871251,  0.46730675,  0.34487649,
        0.01764426,  0.36634706, -0.12739366,  0.07704822,  0.00303196])


Iteration # 2
Eigenvalue: 1.773540
Eigenvector: array([ 0.33765539,  0.10420235, -0.57970023,  0.49313065,  0.11481791,
       -0.31817328,  0.38216597, -0.18311247,  0.00882072,  0.04271208])


Iteration # 3
Eigenvalue: 1.329308
Eigenvector: array([ 0.21175437,  0.07211066, -0.5423908 ,  0.51134725,  0.02029293,
       -0.41778141,  0.42671577, -0.15718928, -0.06919839,  0.08809089])


Iteration # 4
Eigenvalue: 1.253878
Eigenvector: array([ 0.15877645,  0.06100111, -0.49159388,  0.51204839, -0.01859177,
       -0.44765639,  0.46262182, -0.13332977, -0.1348737 ,  0.1283831 ])


Iteration # 5
Eigenvalue: 1.227185
Eigenvector: array([ 0.13640295,  0.05127466, -0.453313  ,  0.50358112, -0.0361393 ,
       -0.45831629,  0.48452497, -0.11644467, -0.18471597,  0.16085662])


Iteration # 6
Eigenvalue:

This gives us an estimated shift of 1.2. Running again:

In [29]:
small_eval,s_evector = inverse_power(A2,1.2,300,tol=1E-6)

Iteration # 1
Eigenvalue: 1.226050
Eigenvector: array([ 0.228165  , -0.22182636, -0.18708891,  0.54061434, -0.33329426,
       -0.23748331,  0.52567358, -0.25502007, -0.14854617,  0.20137123])


Iteration # 2
Eigenvalue: 1.205715
Eigenvector: array([ 0.20334702, -0.16218771, -0.24369216,  0.53354801, -0.2647989 ,
       -0.31167475,  0.53947759, -0.21157345, -0.19623749,  0.21614263])


Iteration # 3
Eigenvalue: 1.205698
Eigenvector: array([ 0.21292231, -0.18737682, -0.22027024,  0.53842687, -0.29965421,
       -0.27732959,  0.53559874, -0.23556887, -0.1711516 ,  0.2068227 ])


Iteration # 4
Eigenvalue: 1.205694
Eigenvector: array([ 0.20846221, -0.17543694, -0.231608  ,  0.53644813, -0.28323636,
       -0.29386348,  0.53779438, -0.22432543, -0.18316053,  0.21136574])


Iteration # 5
Eigenvalue: 1.205693
Eigenvector: array([ 0.21061451, -0.18114491, -0.22624406,  0.53746613, -0.29109208,
       -0.28603408,  0.53682538, -0.22971021, -0.17746886,  0.20923257])


Iteration # 6
Eigenvalue:

In [30]:
print("Smallest eigenvalue using inverse power iterations: %.6f"%small_eval)
print("Corresponding eigenvector:")
pp.pprint(s_evector)

Smallest eigenvalue using inverse power iterations: 1.205693
Corresponding eigenvector:
array([ 0.20992636, -0.17930934, -0.22798012,  0.53715294, -0.28856702,
       -0.28856686,  0.53715292, -0.22798023, -0.17930923,  0.20992632])


Therefore, **inverse power method** now took 19 iterations to converge to the smallest eigenvalue, 1.205693.
We use eigenvalue shifting for rapid convergence.